In [1]:
import pandas as pd

In [2]:
rules = pd.read_json("data/arules.json")
rules["antecedents"] = rules["antecedents"].apply(lambda x: frozenset(x))
rules["consequents"] = rules["consequents"].apply(lambda x: frozenset(x))

In [3]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(IAmA),(AskReddit),0.217120,0.480390,0.127371,0.586641,1.221177,0.023069,1.257043
1,(fffffffuuuuuuuuuuuu),(IAmA),0.003648,0.217120,0.002052,0.562500,2.590737,0.001260,1.789441
2,(todayilearned),(IAmA),0.013818,0.217120,0.008072,0.584158,2.690491,0.005072,1.882641
3,(wikipedia),(IAmA),0.004378,0.217120,0.002280,0.520833,2.398831,0.001330,1.633837
4,(bestof),(IAmA),0.002782,0.217120,0.001687,0.606557,2.793655,0.001083,1.989821
...,...,...,...,...,...,...,...,...,...
102302,"(pics, politics, funny, programming, science, ...","(reddit.com, WTF, technology)",0.002417,0.017968,0.001323,0.547170,30.452639,0.001279,2.168654
102303,"(WTF, politics, funny, programming, science, w...","(reddit.com, pics, technology)",0.002371,0.017785,0.001323,0.557692,31.356607,0.001280,2.220659
102304,"(WTF, pics, politics, funny, programming, worl...","(science, technology, reddit.com)",0.002645,0.014867,0.001323,0.500000,33.631902,0.001283,1.970266
102305,"(worldnews, politics, funny, programming, tech...","(science, reddit.com, WTF, pics)",0.002326,0.021707,0.001323,0.568627,26.195090,0.001272,2.267860


In [4]:
for column in ["support", "confidence", "lift", "leverage", "conviction"]:
#     rules[column] = (rules[column] - rules[column].mean())/rules[column].std()
    rules[column] = (rules[column] - rules[column].min())/(rules[column].max() - rules[column].min())

In [8]:
# OBVIOUS_SUBREDDITS = {'reddit.com', 'AskReddit', 'pics', 'WTF', 'IAmA'}

In [29]:
def get_rules(set_of_subreddits):
    antecedents_rule =  rules['antecedents'].apply(lambda x: set_of_subreddits.issuperset(x))
    new_rules =  rules[antecedents_rule].copy()
#     new_rules["Coefficient"] = new_rules["confidence"] + new_rules["support"]
#     new_rules["Coefficient"] = new_rules["lift"]
    new_rules["Coefficient"] = new_rules["confidence"] - new_rules["support"] + new_rules["lift"] + new_rules["leverage"]+ new_rules["conviction"]
#     new_rules["consequents"] = new_rules["consequents"].apply(lambda x: x - set_of_subreddits - OBVIOUS_SUBREDDITS)
    new_rules["consequents"] = new_rules["consequents"].apply(lambda x: x - set_of_subreddits)
    new_rules = new_rules[new_rules["consequents"].apply(lambda x: len(x) > 0)]
    if len(new_rules) == 0:
        return []
    new_rules = new_rules[['consequents', "Coefficient"]]
    new_rules = new_rules.explode("consequents")
    new_rules["consequents"] = new_rules["consequents"].apply(lambda x:list(x)[0])
    new_rules = new_rules.groupby("consequents")["Coefficient"].max().reset_index()    
    return list(new_rules.nlargest(100, "Coefficient")["consequents"])

In [30]:
%timeit df = get_rules({'funny', 'WTF', 'math', 'fffffffuuuuuuuuuuuu', 'bestof', 'offbeat'})
df

39.1 ms ± 659 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


['pics',
 'reddit.com',
 'AskReddit',
 'IAmA',
 'science',
 'programming',
 'politics']

In [17]:
rules[rules['antecedents'].apply(lambda x: x.issuperset({'math'}))]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
126,"(math, IAmA)",(AskReddit),0.002417,0.480390,0.005423,0.584906,0.007563,0.014939,0.018582
127,"(math, AskReddit)",(IAmA),0.003740,0.217120,0.005423,0.024390,0.016376,0.022327,0.007178
212,"(WTF, math)",(IAmA),0.002098,0.217120,0.000000,0.173913,0.020653,0.015395,0.010865
213,"(math, IAmA)",(WTF),0.002417,0.132889,0.000000,0.018868,0.034693,0.018237,0.009243
214,"(math, IAmA)",(reddit.com),0.002417,0.287121,0.003615,0.396226,0.017275,0.019998,0.016778
493,"(science, math)",(AskReddit),0.002280,0.480390,0.002892,0.400000,0.005172,0.009574,0.008792
525,"(WTF, math)",(AskReddit),0.002098,0.480390,0.002169,0.434783,0.005622,0.009499,0.010144
526,"(reddit.com, math)",(AskReddit),0.003557,0.480390,0.008677,0.307692,0.003978,0.012034,0.005861
527,"(math, AskReddit)",(reddit.com),0.003740,0.287121,0.008677,0.243902,0.013980,0.025473,0.010741
528,"(math, pics)",(AskReddit),0.002052,0.480390,0.001446,0.377778,0.004885,0.008032,0.008007
